In [11]:
# -*- coding: utf-8 -*-

class Dtw(object):
    
    def __init__(self, seq1, seq2,
                 patterns = [(-1,-1), (-1,0), (0,-1)], 
                 weights = [{(0,0):2}, {(0,0):1}, {(0,0):1}], 
                 band_r=0.5):
        self._seq1 = seq1
        self._seq2 = seq2
        self._r = min(10, band_r*max(len(seq1), len(seq2)))
        assert len(patterns) == len(weights)
        self._patterns = patterns
        self._weights = weights
        self._map = {(-1, -1): 0}
    
    def get_distance(self, idx1, idx2):
        if idx1<0 or idx2<0 or idx1>=len(self._seq1) or idx2>=len(self._seq2):
            return 0
        return abs(self._seq1[idx1] - self._seq2[idx2])

    def calculate_path(self, idx1, idx2, pattern, weight):
        g = self.calculate(idx1+pattern[0], idx2+pattern[1])
        sum_d = 0
        for coor_offset, w in weight.items():
            i1, i2 = map(sum, zip((idx1, idx2), coor_offset))
            sum_d += self.get_distance(i1, i2)
        return g + sum_d

    def calculate(self, idx1, idx2):
        if (idx1, idx2) in self._map:
            return self._map[(idx1, idx2)]
        if idx1 < 0 or idx2 < 0 or abs(idx1-idx2) > self._r:
            return float('inf')
        min_prev_dp = float('inf')
        for i in range(len(self._patterns)):
            min_prev_dp = min(min_prev_dp, self.calculate_path(idx1, idx2, self._patterns[i], self._weights[i]))
        self._map[(idx1, idx2)] = min_prev_dp
        return self._map[(idx1, idx2)]
    
    @property
    def dtw_matrix_dict(self):
        return self._map

    def print_dtw_matrix(self):
        print('      '+' '.join(["{:^7d}".format(i) for i in range(len(self._seq2))]))
        for i in range(len(self._seq1)):
            str = "{:^4d}: ".format(i)
            for j in range(len(self._seq2)):
                if (i,j) not in self._map:
                    str += "{:^7s} ".format('-')
                else:
                    str += "{:^7.3f} ".format(self._map[(i,j)])
            print (str)
    
    def get_dtw(self):
        g = self.calculate(len(self._seq1)-1, len(self._seq2)-1)
        N = len(self._seq1) + len(self._seq2)
        return g/N

# Main

### Different patterns

In [20]:
PATTERNS_1 = [(0,-1), (-1,-1), (-1,0)]
WEIGHTS_SYM_1 = [{(0,0):1}, {(0,0):2}, {(0,0):1}] 
WEIGHTS_ASYM_1 = [{}, {(0,0):1}, {(0,0):1}] 
WEIGHTS_ASYM_1_2 = [{(0,0):100}, {(0,0):10}, {(0,0):1}] 

PATTERNS_2 = [(-1,-3), (-1,-2), (-1,-1), (-2,-1), (-3,-1)]
WEIGHTS_SYM_2 = [{(0,-2):2, (0,-1):1, (0,0):1}, \
                 {(0,-1):2, (0,0):1}, \
                 {(0,0):2}, \
                 {(-1,0):2, (0,0):1}, \
                 {(-2,0):2, (-1,0):1, (0,0):1}] 
WEIGHTS_ASYM_2 = [{(0,-2):1, (0,-1):1, (0,0):1/3}, \
                  {(0,-1):1, (0,0):1/2}, \
                  {(0,0):1}, \
                  {(-1,0):1, (0,0):1}, \
                  {(-2,0):1, (-1,0):1, (0,0):1}] 

### Initiation

In [20]:
import numpy as np
seq1 = [1, 1, 2, 9]*2
seq2 = [0, 1, 1, 2]*2

### Z-Normalization

In [14]:
seq1 = (np.array(seq1)-np.mean(seq1))/np.std(seq1)
seq2 = (np.array(seq2)-np.mean(seq2))/np.std(seq2)

### Calculate DTW

#### Symmetric Pattern 1
g(i, j) = min( g(i,j-1)+d(i,j),  g(i-1,j-1)+2d(i,j),  g(i-1,j)+d(i,j) )

In [15]:
d = Dtw(seq1, seq2, PATTERNS_1, WEIGHTS_SYM_1)
d.get_dtw()

0.26160228246317752

In [16]:
d.print_dtw_matrix()

         0       1       2       3       4       5       6       7   
 0  :  0.742   1.414   2.087   4.174   4.915     -       -       -    
 1  :  1.483   1.414   2.087   4.174   4.915   5.588     -       -    
 2  :  2.524   1.788   1.788   3.576   4.616   4.990   5.364     -    
 3  :  5.657   3.507   3.507   2.093   5.226   6.335   6.709   5.669  
 4  :  6.398   4.180   4.180   4.180   2.834   3.507   4.180   6.267  
 5  :    -     4.852   4.852   6.267   3.576   3.507   4.180   6.267  
 6  :    -       -     5.226   6.640   4.616   3.881   3.881   5.669  
 7  :    -       -       -     5.531   7.750   5.600   5.600   4.186  


#### Asymmetric Pattern 2
g(i, j) = min( g(i,j-1),  g(i-1,j-1)+d(i,j),  g(i-1,j)+d(i,j) )

In [7]:
d = Dtw(seq1, seq2, PATTERNS_1, WEIGHTS_ASYM_1)
d.get_dtw()

0.25730038274945188

In [10]:
d.print_dtw_matrix()

NameError: name 'd' is not defined

#### Symmetric Pattern 2

In [9]:
d = Dtw(seq1, seq2, PATTERNS_2, WEIGHTS_SYM_2)
d.get_dtw()

0.26160228246317752

In [10]:
d.print_dtw_matrix()

         0       1       2       3       4       5       6       7   
 0  :  0.742   1.414   2.087    inf     inf      -       -       -    
 1  :  1.483   1.414   2.087   4.174   4.915   5.588     -       -    
 2  :  2.455   1.788   1.788   3.576   4.616   5.289     -       -    
 3  :   inf    3.507   3.507   2.093   5.226   6.335     -       -    
 4  :   inf    4.249   2.766   4.180   2.834   3.507   4.180     -    
 5  :    -     5.519   4.852   4.852   3.576   3.507   4.180     -    
 6  :    -       -       -       -     4.548   3.881   3.881     -    
 7  :    -       -       -       -       -       -       -     4.186  


#### Asymmetric Pattern 2

In [11]:
d = Dtw(seq1, seq2, PATTERNS_2, WEIGHTS_ASYM_2)
d.get_dtw()

0.26160228246317752

In [12]:
d.print_dtw_matrix()

         0       1       2       3       4       5       6       7   
 0  :  0.742   1.414   2.087    inf     inf      -       -       -    
 1  :  1.483   1.414   2.087   4.174   4.915   5.588     -       -    
 2  :  2.455   1.788   1.788   3.576   4.616   5.289     -       -    
 3  :   inf    3.507   3.507   2.093   5.226   6.335     -       -    
 4  :   inf    4.249   2.766   4.180   2.834   3.507   4.180     -    
 5  :    -     5.519   4.852   4.852   3.576   3.507   4.180     -    
 6  :    -       -       -       -     4.548   3.881   3.881     -    
 7  :    -       -       -       -       -       -       -     4.186  


# Implementation2

In [24]:
# -*- coding: utf-8 -*-

class Dtw(object):
    
    def __init__(self, seq1, seq2,
                 patterns = [(-1,-1), (-1,0), (0,-1)], 
                 weights = [{(0,0):2}, {(0,0):1}, {(0,0):1}], 
                 band_r=0.1):
        self._seq1 = seq1
        self._seq2 = seq2
        self.len_seq1 = len(seq1)
        self.len_seq2 = len(seq2)
        self.len_pattern = len(patterns)
        self.sum_w = [sum(ws.values()) for ws in weights]
        self._r = len(seq1)//band_r
        assert len(patterns) == len(weights)
        self._patterns = patterns
        self._weights = weights
    
    def get_distance(self, i1, i2):
        return abs(self._seq1[i1] - self._seq2[i2])

    def calculate(self):
        g = list([float('inf')]*self.len_seq2 for i in range(self.len_seq1))
        cost = list([0]*self.len_seq2 for i in range(self.len_seq1))

        g[0][0] = 2*self.get_distance(0, 0)
        for i in range(self.len_seq1):
            for j in range(max(0,i-self._r), min(i+self._r+1, self.len_seq2)):
                for pat_i in range(self.len_pattern):
                    coor = (i+self._patterns[pat_i][0], j+self._patterns[pat_i][1])
                    if coor[0]<0 or coor[1]<0:
                        continue
                    dist = 0
                    for w_coor_offset, d_w in self._weights[pat_i].items():
                        w_coor = (i+w_coor_offset[0], j+w_coor_offset[1])
                        dist += d_w*self.get_distance(w_coor[0], w_coor[1])
                    this_val = g[coor[0]][coor[1]] + dist
                    this_cost = cost[coor[0]][coor[1]] + self.sum_w[pat_i]
                    if this_val < g[i][j]:
                        g[i][j] = this_val
                        cost[i][j] = this_cost
        return g[self.len_seq1-1][self.len_seq2-1]/cost[self.len_seq1-1][self.len_seq2-1], g, cost
    
    def print_table(self, tb):
        print('      '+' '.join(["{:^7d}".format(i) for i in range(self.len_seq2)]))
        for i in range(self.len_seq1):
            str = "{:^4d}: ".format(i)
            for j in range(self.len_seq2):
                str += "{:^7.3f} ".format(tb[i][j])
            print (str)

    def print_g_matrix(self):
        _, tb, _ = self.calculate()
        self.print_table(tb)

    def print_cost_matrix(self):
        _, _, tb = self.calculate()
        self.print_table(tb)
        
    def get_dtw(self):
        ans, _, _ = self.calculate()
        return ans

In [25]:
import numpy as np
seq1 = [1, 1, 2, 9]*2
seq2 = [0, 1, 1, 2]*2
seq1 = (np.array(seq1)-np.mean(seq1))/np.std(seq1)
seq2 = (np.array(seq2)-np.mean(seq2))/np.std(seq2)

In [26]:
d = Dtw(seq1, seq2, PATTERNS_1, WEIGHTS_ASYM_1)
d.get_dtw()

0.69404687184108327

In [27]:
d.print_g_matrix()

         0       1       2       3       4       5       6       7   
 0  :  1.483   1.483   1.483   1.483   1.483   1.483   1.483   1.483  
 1  :  2.225   2.156   2.156   2.156   2.156   2.156   2.156   2.156  
 2  :  3.265   2.529   2.529   2.529   2.529   2.529   2.529   2.529  
 3  :  6.398   4.249   4.249   2.834   2.834   2.834   2.834   2.834  
 4  :  7.140   4.921   4.921   4.921   3.576   3.507   3.507   3.507  
 5  :  7.881   5.594   5.594   5.594   4.317   4.180   4.180   4.180  
 6  :  8.922   5.968   5.968   5.968   5.358   4.553   4.553   4.553  
 7  : 12.055   7.687   7.687   6.273   6.273   6.273   6.273   4.858  


In [28]:
d.print_cost_matrix()

         0       1       2       3       4       5       6       7   
 0  :  0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000  
 1  :  1.000   1.000   1.000   1.000   1.000   1.000   1.000   1.000  
 2  :  2.000   2.000   2.000   2.000   2.000   2.000   2.000   2.000  
 3  :  3.000   3.000   3.000   3.000   3.000   3.000   3.000   3.000  
 4  :  4.000   4.000   4.000   4.000   4.000   4.000   4.000   4.000  
 5  :  5.000   5.000   5.000   5.000   5.000   5.000   5.000   5.000  
 6  :  6.000   6.000   6.000   6.000   6.000   6.000   6.000   6.000  
 7  :  7.000   7.000   7.000   7.000   7.000   7.000   7.000   7.000  


## Test another series

In [32]:
vec1=[1,7,2,3,6,9,4,1]
vec2=[1,8,8,8,2,7,7,7,0]
Dtw(vec1,vec2,PATTERNS_1,WEIGHTS_SYM_1).print_g_matrix()

         0       1       2       3       4       5       6       7       8   
 0  :  0.000   7.000  14.000  21.000  22.000  28.000  34.000  40.000  41.000  
 1  :  6.000   2.000   3.000   4.000   9.000   9.000   9.000   9.000  16.000  
 2  :  7.000   8.000   9.000  10.000   4.000   9.000  14.000  14.000  13.000  
 3  :  9.000  13.000  14.000  15.000   5.000   9.000  13.000  17.000  16.000  
 4  : 14.000  13.000  15.000  17.000   9.000   7.000   8.000   9.000  15.000  
 5  : 22.000  14.000  15.000  16.000  16.000   9.000  10.000  11.000  20.000  
 6  : 25.000  18.000  19.000  20.000  18.000  12.000  13.000  14.000  18.000  
 7  : 25.000  25.000  26.000  27.000  19.000  18.000  19.000  20.000  16.000  


In [33]:
Dtw(vec1,vec2,PATTERNS_1,WEIGHTS_SYM_1).print_cost_matrix()

         0       1       2       3       4       5       6       7       8   
 0  :  0.000   1.000   2.000   3.000   4.000   5.000   6.000   7.000   8.000  
 1  :  1.000   2.000   3.000   4.000   5.000   6.000   7.000   8.000   9.000  
 2  :  2.000   3.000   4.000   5.000   6.000   7.000   8.000   9.000  10.000  
 3  :  3.000   4.000   5.000   6.000   7.000   8.000   9.000  10.000  11.000  
 4  :  4.000   5.000   6.000   7.000   8.000   9.000  10.000  11.000  12.000  
 5  :  5.000   6.000   7.000   8.000   9.000  10.000  11.000  12.000  13.000  
 6  :  6.000   7.000   8.000   9.000  10.000  11.000  12.000  13.000  14.000  
 7  :  7.000   8.000   9.000  10.000  11.000  12.000  13.000  14.000  15.000  


In [36]:
Dtw(vec1,vec2,PATTERNS_1,WEIGHTS_SYM_1).get_dtw()

1.0666666666666667

In [34]:
Dtw(vec1, vec2, PATTERNS_1, WEIGHTS_ASYM_1_2).print_g_matrix()

         0       1       2       3       4       5       6       7       8   
 0  :  0.000  700.000 1400.000 2100.000 2200.000 2800.000 3400.000 4000.000 4100.000 
 1  :  6.000  10.000  110.000 210.000 710.000 710.000 710.000 710.000 1410.000 
 2  :  7.000  16.000  70.000  170.000 170.000 670.000 715.000 715.000 730.000 
 3  :  9.000  21.000  66.000  120.000 171.000 210.000 610.000 719.000 733.000 
 4  : 14.000  23.000  41.000  86.000  160.000 181.000 220.000 320.000 739.000 
 5  : 22.000  24.000  33.000  51.000  156.000 180.000 201.000 240.000 410.000 
 6  : 25.000  28.000  37.000  55.000  71.000  183.000 204.000 231.000 280.000 
 7  : 25.000  35.000  44.000  62.000  65.000  131.000 210.000 237.000 241.000 


In [35]:
Dtw(vec1, vec2, PATTERNS_1, WEIGHTS_ASYM_1_2).print_cost_matrix()

         0       1       2       3       4       5       6       7       8   
 0  :  0.000  100.000 200.000 300.000 400.000 500.000 600.000 700.000 800.000 
 1  :  1.000  10.000  110.000 210.000 310.000 410.000 510.000 610.000 710.000 
 2  :  2.000  11.000  20.000  120.000 220.000 320.000 511.000 611.000 620.000 
 3  :  3.000  12.000  21.000  30.000  221.000 230.000 330.000 612.000 621.000 
 4  :  4.000  13.000  22.000  31.000  40.000  231.000 240.000 340.000 622.000 
 5  :  5.000  14.000  23.000  32.000  41.000  50.000  241.000 250.000 350.000 
 6  :  6.000  15.000  24.000  33.000  42.000  51.000  242.000 251.000 260.000 
 7  :  7.000  16.000  25.000  34.000  43.000  52.000  243.000 252.000 261.000 


In [37]:
Dtw(vec1,vec2,PATTERNS_1,WEIGHTS_ASYM_1_2).get_dtw()

0.9233716475095786